In [1]:
import numpy as np
import pandas as pd

# train_xは学習データ、train_yは目的変数、test_xはテストデータ
# pandasのDataFrame, Seriesで保持します。（numpyのarrayで保持することもあります）

train = pd.read_csv('../input/sample-data/train_preprocessed.csv')
train_x = train.drop(['target'], axis=1)
train_y = train['target']
test_x = pd.read_csv('../input/sample-data/test_preprocessed.csv')

# neural net用のデータ
train_nn = pd.read_csv('../input/sample-data/train_preprocessed_onehot.csv')
train_x_nn = train_nn.drop(['target'], axis=1)
train_y_nn = train_nn['target']
test_x_nn = pd.read_csv('../input/sample-data/test_preprocessed_onehot.csv')

In [2]:
train

,age,sex,height,weight,product,amount,medical_info_a1,medical_info_a2,medical_info_a3,medical_info_b1,...,medical_keyword_6,medical_keyword_7,medical_keyword_8,medical_keyword_9,medical_keyword_10,year,month,day,yearmonth,target
0,50,1,166.445608,65.016732,9,7000000,134,202,1,11,...,1,0,1,0,0,2015,2,3,24182,0
1,68,0,164.334615,56.544217,0,7000000,438,263,3,14,...,0,1,1,0,0,2015,5,9,24185,0
2,77,1,167.462917,54.242267,2,6000000,313,325,1,18,...,1,0,1,0,0,2016,2,13,24194,1
3,17,1,177.097725,71.147762,3,8000000,342,213,2,11,...,0,0,1,0,0,2015,7,6,24187,0
4,62,0,158.165788,65.240697,1,9000000,327,102,0,14,...,0,1,1,1,0,2016,9,17,24201,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,61,1,182.729800,73.393777,1,2000000,189,232,7,17,...,0,0,1,1,0,2015,10,21,24190,0
9996,33,0,167.701136,75.006529,8,9000,426,202,3,19,...,0,0,1,1,0,2015,5,28,24185,0
9997,44,0,145.609998,47.739397,8,1000,370,274,1,11,...,0,0,1,0,1,2016,2,29,24194,0
9998,34,0,165.796017,57.567695,6,5000,291,105,1,13,...,1,1,1,1,0,2016,2,27,24194,0


In [6]:
train_nn

,age,sex,height,weight,product_0,product_1,product_2,product_3,product_4,product_5,...,medical_keyword_8,medical_keyword_9,medical_keyword_10,year,month,day,yearmonth,medical_info_c1_nan,medical_info_c2_nan,target
0,50,1,166.445608,65.016732,0,0,0,0,0,0,...,1,0,0,2015,2,3,24182,False,True,0
1,68,0,164.334615,56.544217,1,0,0,0,0,0,...,1,0,0,2015,5,9,24185,True,True,0
2,77,1,167.462917,54.242267,0,0,1,0,0,0,...,1,0,0,2016,2,13,24194,False,True,1
3,17,1,177.097725,71.147762,0,0,0,1,0,0,...,1,0,0,2015,7,6,24187,False,False,0
4,62,0,158.165788,65.240697,0,1,0,0,0,0,...,1,1,0,2016,9,17,24201,False,True,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,61,1,182.729800,73.393777,0,1,0,0,0,0,...,1,1,0,2015,10,21,24190,False,True,0
9996,33,0,167.701136,75.006529,0,0,0,0,0,0,...,1,1,0,2015,5,28,24185,False,True,0
9997,44,0,145.609998,47.739397,0,0,0,0,0,0,...,1,0,1,2016,2,29,24194,True,True,0
9998,34,0,165.796017,57.567695,0,0,0,0,0,0,...,1,1,0,2016,2,27,24194,False,True,0


## スタッキング

In [7]:
from sklearn.metrics import log_loss
from sklearn.model_selection import KFold

# models.pyにModel1Xgb, Model1NN, Model2Linearを定義しているものとする
# 各クラスは、fitで学習し、predictで予測値の確率を出力する

from models import Model1Xgb, Model1NN, Model2Linear

In [8]:
# 学習データに対する「目的変数を知らない」予測値と、テストデータに対する予測値を返す関数
def predict_cv(model, train_x, train_y, test_x):
    preds = []
    preds_test = []
    va_idxes = []

    kf = KFold(n_splits=4, shuffle=True, random_state=71)

    # クロスバリデーションで学習・予測を行い、予測値とインデックスを保存する
    for i, (tr_idx, va_idx) in enumerate(kf.split(train_x)):
        tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
        tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]
        model.fit(tr_x, tr_y, va_x, va_y)
        pred = model.predict(va_x)
        preds.append(pred)
        pred_test = model.predict(test_x)
        preds_test.append(pred_test)
        va_idxes.append(va_idx)

    # バリデーションデータに対する予測値を連結し、その後元の順序に並べ直す
    va_idxes = np.concatenate(va_idxes)
    preds = np.concatenate(preds, axis=0)
    order = np.argsort(va_idxes)
    pred_train = preds[order]

    # テストデータに対する予測値の平均をとる
    preds_test = np.mean(preds_test, axis=0)

    return pred_train, preds_test

In [9]:
# 1層目のモデル
# pred_train_1a, pred_train_1bは、学習データのクロスバリデーションでの予測値
# pred_test_1a, pred_test_1bは、テストデータの予測値
model_1a = Model1Xgb()
pred_train_1a, pred_test_1a = predict_cv(model_1a, train_x, train_y, test_x)

model_1b = Model1NN()
pred_train_1b, pred_test_1b = predict_cv(model_1b, train_x_nn, train_y, test_x_nn)

# 1層目のモデルの評価
print(f'logloss: {log_loss(train_y, pred_train_1a, eps=1e-7):.4f}')
print(f'logloss: {log_loss(train_y, pred_train_1b, eps=1e-7):.4f}')

[11:59:58] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-logloss:0.54088	eval-logloss:0.55003
[1]	train-logloss:0.45269	eval-logloss:0.47182
[2]	train-logloss:0.39482	eval-logloss:0.42026
[3]	train-logloss:0.35198	eval-logloss:0.38520
[4]	train-logloss:0.32021	eval-logloss:0.36150
[5]	train-logloss:0.29673	eval-logloss:0.34463
[6]	train-logloss:0.27610	eval-logloss:0.32900
[7]	train-logloss:0.25886	eval-logloss:0.31670
[8]	train-logloss:0.24363	eval-logloss:0.30775
[9]	train-logloss:0.23153	eval-logloss:0.30092
[11:59:58] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "silent" } might not be used

C:\Users\s44990\AppData\Roaming\Python\Python39\site-packages\keras\engine\sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '
C:\Users\s44990\AppData\Roaming\Python\Python39\site-packages\keras\engine\sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


Epoch 1/10
59/59 [==============================] - 2s 7ms/step - loss: 0.5302 - val_loss: 0.3736
Epoch 2/10
59/59 [==============================] - 0s 3ms/step - loss: 0.3523 - val_loss: 0.3679
Epoch 3/10
59/59 [==============================] - 0s 6ms/step - loss: 0.3544 - val_loss: 0.3634
Epoch 4/10
59/59 [==============================] - 0s 4ms/step - loss: 0.3244 - val_loss: 0.3501
Epoch 5/10
59/59 [==============================] - 0s 4ms/step - loss: 0.3042 - val_loss: 0.3346
Epoch 6/10
59/59 [==============================] - 0s 4ms/step - loss: 0.2773 - val_loss: 0.3130
Epoch 7/10
59/59 [==============================] - 0s 4ms/step - loss: 0.2379 - val_loss: 0.2855
Epoch 8/10
59/59 [==============================] - 0s 4ms/step - loss: 0.2219 - val_loss: 0.2766
Epoch 9/10
59/59 [==============================] - 0s 4ms/step - loss: 0.2045 - val_loss: 0.2681
Epoch 10/10
59/59 [==============================] - 0s 4ms/step - loss: 0.1822 - val_loss: 0.2642


C:\Users\s44990\AppData\Roaming\Python\Python39\site-packages\keras\engine\sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '
C:\Users\s44990\AppData\Roaming\Python\Python39\site-packages\keras\engine\sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


Epoch 1/10
59/59 [==============================] - 1s 7ms/step - loss: 0.5004 - val_loss: 0.3729
Epoch 2/10
59/59 [==============================] - 0s 3ms/step - loss: 0.3748 - val_loss: 0.3559
Epoch 3/10
59/59 [==============================] - 0s 3ms/step - loss: 0.3433 - val_loss: 0.3474
Epoch 4/10
59/59 [==============================] - 0s 3ms/step - loss: 0.3358 - val_loss: 0.3406
Epoch 5/10
59/59 [==============================] - 0s 3ms/step - loss: 0.3091 - val_loss: 0.3299
Epoch 6/10
59/59 [==============================] - 0s 3ms/step - loss: 0.2882 - val_loss: 0.3256
Epoch 7/10
59/59 [==============================] - 0s 3ms/step - loss: 0.2731 - val_loss: 0.2980
Epoch 8/10
59/59 [==============================] - 0s 3ms/step - loss: 0.2243 - val_loss: 0.2888
Epoch 9/10
59/59 [==============================] - 0s 3ms/step - loss: 0.2108 - val_loss: 0.2734
Epoch 10/10
59/59 [==============================] - 0s 3ms/step - loss: 0.1979 - val_loss: 0.2749


C:\Users\s44990\AppData\Roaming\Python\Python39\site-packages\keras\engine\sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '
C:\Users\s44990\AppData\Roaming\Python\Python39\site-packages\keras\engine\sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


Epoch 1/10
59/59 [==============================] - 1s 8ms/step - loss: 0.4888 - val_loss: 0.3658
Epoch 2/10
59/59 [==============================] - 0s 3ms/step - loss: 0.3587 - val_loss: 0.3622
Epoch 3/10
59/59 [==============================] - 0s 3ms/step - loss: 0.3435 - val_loss: 0.3514
Epoch 4/10
59/59 [==============================] - 0s 3ms/step - loss: 0.3249 - val_loss: 0.3415
Epoch 5/10
59/59 [==============================] - 0s 3ms/step - loss: 0.2974 - val_loss: 0.3387
Epoch 6/10
59/59 [==============================] - 0s 3ms/step - loss: 0.2680 - val_loss: 0.3225
Epoch 7/10
59/59 [==============================] - 0s 3ms/step - loss: 0.2391 - val_loss: 0.2930
Epoch 8/10
59/59 [==============================] - 0s 3ms/step - loss: 0.2040 - val_loss: 0.2966
Epoch 9/10
59/59 [==============================] - 0s 3ms/step - loss: 0.1880 - val_loss: 0.2925
Epoch 10/10
59/59 [==============================] - 0s 3ms/step - loss: 0.1755 - val_loss: 0.2964


C:\Users\s44990\AppData\Roaming\Python\Python39\site-packages\keras\engine\sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '
C:\Users\s44990\AppData\Roaming\Python\Python39\site-packages\keras\engine\sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


logloss: 0.2967
logloss: 0.2812


In [10]:
# 予測値を特徴量としてデータフレームを作成
train_x_2 = pd.DataFrame({'pred_1a': pred_train_1a, 'pred_1b': pred_train_1b})
test_x_2 = pd.DataFrame({'pred_1a': pred_test_1a, 'pred_1b': pred_test_1b})

In [11]:
# 2層目のモデル
# pred_train_2は、2層目のモデルの学習データのクロスバリデーションでの予測値
# pred_test_2は、2層目のモデルのテストデータの予測値
model_2 = Model2Linear()
pred_train_2, pred_test_2 = predict_cv(model_2, train_x_2, train_y, test_x_2)
print(f'logloss: {log_loss(train_y, pred_train_2, eps=1e-7):.4f}')

logloss: 0.2495


In [13]:
a = np.arange(20).reshape(4,5)
np.concatenate(a)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19])

In [17]:
a = np.array([1,0,2,6,2,8,6])
np.argsort(a)

array([1, 0, 2, 4, 3, 6, 5], dtype=int64)